In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchain-openai faiss-cpu
%pip install --upgrade --quiet pysqlite3 openai langchain langchain-community langchain-experimental

In [ ]:
import os
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GROQ')

In [ ]:
from google.colab import drive
drive.mount("/content/data")

Mounted at /content/data


In [ ]:
drive_path = "/content/data/MyDrive/sql-data"

In [ ]:
csv_files = os.listdir(drive_path)
csv_files

['people_info.csv',
 'event_info.csv',
 'company_info.csv',
 'event_company_people.db']

In [ ]:
csv_file_paths = [os.path.join(drive_path,i) for i in csv_files]
csv_file_paths

['/content/data/MyDrive/sql-data/people_info.csv',
 '/content/data/MyDrive/sql-data/event_info.csv',
 '/content/data/MyDrive/sql-data/company_info.csv',
 '/content/data/MyDrive/sql-data/event_company_people.db']

# Company_data Preprocessing

In [ ]:
company_df = pd.read_csv(csv_file_paths[2])
company_df.head(2)

,company_logo_url,company_logo_text,company_name,relation_to_event,event_url,company_revenue,n_employees,company_phone,company_founding_year,company_address,company_industry,company_overview,homepage_url,linkedin_company_url,homepage_base_url,company_logo_url_on_event_page,company_logo_match_flag
0,https://d1hbpr09pwz0sk.cloudfront.net/logo_url...,Women in Finance,100 Women In Finance,partner,https://apac.commoditytradingweek.com/,NaN,11-50,NaN,2001.0,NaN,Financial Services,100 Women in Finance strengthens the global fi...,https://100women.org/events/,https://www.linkedin.com/company/100-women-in-...,100women.org,https://apac.commoditytradingweek.com/wp-conte...,yes
1,https://media.licdn.com/dms/image/C4D0BAQHlTYA...,BBGC,BBGC,sponsor,https://apac.commoditytradingweek.com/,NaN,51-200,NaN,NaN,NaN,IT Services and IT Consulting,Business Benefits Global Consulting (BBGC) is ...,www.bbgcservices.com,https://it.linkedin.com/company/bbgcservices/a...,bbgcservices.com,https://apac.commoditytradingweek.com/wp-conte...,yes


In [ ]:
company_df.columns[company_df.isnull().any()]

Index(['company_logo_url', 'company_revenue', 'n_employees', 'company_phone',
       'company_founding_year', 'company_address', 'company_industry',
       'company_overview', 'homepage_url', 'linkedin_company_url',
       'company_logo_match_flag'],
      dtype='object')

In [ ]:
company_df.notna().astype(int)

,company_logo_url,company_logo_text,company_name,relation_to_event,event_url,company_revenue,n_employees,company_phone,company_founding_year,company_address,company_industry,company_overview,homepage_url,linkedin_company_url,homepage_base_url,company_logo_url_on_event_page,company_logo_match_flag
0,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,1,1
1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0
738,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1,1,0
739,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0
740,0,1,1,1,1,0,1,0,1,0,1,0,1,1,1,1,0


In [ ]:


def preprocess_company_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # 1. Handle missing values
    df['company_revenue'] = df['company_revenue'].fillna('Unknown')
    df['n_employees'] = df['n_employees'].fillna('Unknown')
    df['company_phone'] = df['company_phone'].fillna('N/A')
    df['company_founding_year'] = df['company_founding_year'].fillna(0)
    df['company_address'] = df['company_address'].fillna('N/A')
    df['company_industry'] = df['company_industry'].fillna('Unknown')
    df['company_overview'] = df['company_overview'].fillna('No overview available')


    df['company_revenue_standardized'] = df['company_revenue'].apply(standardize_revenue)

    df['n_employees_standardized'] = df['n_employees'].apply(standardize_employees)

    df['company_phone_cleaned'] = df['company_phone'].apply(clean_phone)

    # 5. Validate and clean company_founding_year
    current_year = datetime.now().year
    df['company_founding_year'] = pd.to_numeric(df['company_founding_year'], errors='coerce')
    df.loc[df['company_founding_year'] > current_year, 'company_founding_year'] = 0
    df.loc[df['company_founding_year'] < 1800, 'company_founding_year'] = 0

    # 6. Standardize company_industry
    df['company_industry'] = df['company_industry'].str.lower().str.strip()

    # 7. Create a binary flag for logo presence
    df['has_logo'] = df['company_logo_url'].notna().astype(int)

    # 8. Extract domain from homepage_url
    df['domain'] = df['homepage_url'].str.extract(r'https?://(?:www\.)?([^/]+)')

    # 9. Create a binary flag for LinkedIn presence
    df['has_linkedin'] = df['linkedin_company_url'].notna().astype(int)

    # 10. Validate homepage_base_url
    df['valid_homepage_base_url'] = df['homepage_base_url'].str.match(r'^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$').astype(int)

    return df

# Usage
preprocessed_df = preprocess_company_data(csv_file_paths[2])
preprocessed_df.to_csv('preprocessed_company_info.csv', index=False)

print(preprocessed_df.head())
print(preprocessed_df.info())

                                    company_logo_url company_logo_text  \
0  https://d1hbpr09pwz0sk.cloudfront.net/logo_url...  Women in Finance   
1  https://media.licdn.com/dms/image/C4D0BAQHlTYA...              BBGC   
2  https://d1hbpr09pwz0sk.cloudfront.net/logo_url...       HR MARITIME   
3  https://d1hbpr09pwz0sk.cloudfront.net/logo_url...       HR MARITIME   
4  https://d1hbpr09pwz0sk.cloudfront.net/logo_url...  Kerry Consulting   

                  company_name relation_to_event  \
0         100 Women In Finance           partner   
1                         BBGC           sponsor   
2                  HR Maritime           partner   
3  HR Maritime Consultants Ltd           partner   
4             Kerry Consulting           sponsor   

                                event_url company_revenue n_employees  \
0  https://apac.commoditytradingweek.com/         Unknown       11-50   
1  https://apac.commoditytradingweek.com/         Unknown      51-200   
2  https://apac.commodi

In [ ]:
d = "5,001-10,000"
d.split("-")[0].split(" ")[0]

'5,001'

In [ ]:

# 2. Standardize company_revenue
def standardize_revenue(revenue):
  if isinstance(revenue, str):
      if 'million' in revenue.lower():
          return float(re.findall(r'\d+\.?\d*', revenue)[0]) * 1e6
      elif 'billion' in revenue.lower():
          return float(re.findall(r'\d+\.?\d*', revenue)[0]) * 1e9
      elif revenue.lower() == 'unknown':
          return np.nan
  return revenue

 # 3. Standardize n_employees
def standardize_employees(employees):
  if isinstance(employees, str):
      if '-' in employees:
          low, high = map(lambda x: float(x.replace(',', '')), map(lambda y:y.split(" ")[0],employees.split('-')))
          return (low + high) / 2
      elif '+' in employees:
          return float(employees.replace(',', '').replace('+', ''))
      elif employees.lower() == 'unknown':
          return np.nan
  return employees

# 4. Clean and standardize company_phone
def clean_phone(phone):
  if phone == 'N/A':
      return phone
  return re.sub(r'\D', '', str(phone))


# Event Data Preprocessing

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.1 MB/s eta 0:00:00


In [ ]:
from groq import Groq

In [ ]:
# Hypothetical LLM function for tagging event industry
# def llm_tag_event_industry(event_name, event_description):
    # This is a placeholder function. In a real scenario, this would call an LLM API
    # to analyze the event name and description and return an industry tag.
    # return "Technology"  # Placeholder return

def llm_tag_event_industry(event_name, event_description):
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY") or API_KEY,
    )

    prompt = f"""
    Given the following event name and description, determine the most appropriate industry category for this event.
    Choose from the following categories: Technology, Finance, Healthcare, Education, Entertainment, Sports, Business,
    Environment, Hospitality, Politics, Arts, Science, Manufacturing, Retail, Real Estate, Agriculture, Energy, Automotive, Travel,
    Media, Non-profit.

    If the event doesn't clearly fit into one of these categories, choose the closest match or respond with "Other".

    Event Name: {event_name}
    Event Description: {event_description}

    Respond with only the category name, nothing else.
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant specialized in categorizing events into industry sectors based on their names and descriptions.",
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    industry_tag = chat_completion.choices[0].message.content.strip()

    return industry_tag


event_name = "TechCrunch Disrupt 2024"
event_description = "TechCrunch Disrupt is the world's leading authority in debuting revolutionary startups, introducing game-changing technologies, and discussing what's top of mind for the tech industry's key innovators. Disrupt gathers the best and brightest entrepreneurs, investors, hackers, and tech fans for on-stage interviews, the Startup Battlefield competition, a 24-hour Hackathon, Startup Alley, Hardware Alley, and After Parties."

industry = llm_tag_event_industry(event_name, event_description)
print(f"The industry tag for '{event_name}' is: {industry}")

The industry tag for 'TechCrunch Disrupt 2024' is: Technology


In [ ]:
def llm_event_country(event_venue,event_url):
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY") or API_KEY,
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
            "role": "system",
            "content": """YOU ARE A GEOLOGICAL EXPERT AND CAN MAP ANY PLACE TO ITS COUNTRY (CHECK IN THE URL IF THE PLACE IS NOT CLEAR) RETURN JUST THE COUNTRY NAME.
             IF THER IS NO SUCH PLACE RETURN EMPTY STRING"""
            },
            {
                "role": "user",
                "content": "The place name is "+event_venue+" and this is the url related to this : "+event_url,
            }
        ],
        model="llama3-8b-8192",
    )

    event_country = chat_completion.choices[0].message.content.strip()

    return event_country


event_venue = "Sands Expo & Convention Centre"
event_url = "https://www.asiabusinessshow.com/"

country = llm_event_country(event_venue,event_url)
print(f"The industry tag for '{event_venue}' is: {country}")

The industry tag for 'Sands Expo & Convention Centre' is: Singapore


In [ ]:
def preprocess_event_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # 1. Handle missing values
    df['event_logo_url'] = df['event_logo_url'].fillna('N/A')
    df['event_venue'] = df['event_venue'].fillna('Unknown')
    df['event_country'] = df['event_country'].fillna('Unknown')
    df['event_description'] = df['event_description'].fillna('No description available')

    # 2. Parse and validate dates
    df['event_start_date'] = pd.to_datetime(df['event_start_date'], errors='coerce')
    df['event_end_date'] = pd.to_datetime(df['event_end_date'], errors='coerce')

    # 3. Calculate event duration
    df['event_duration'] = (df['event_end_date'] - df['event_start_date']).dt.days + 1

    # 4. Standardize country names
    df['event_country'] = df['event_country'].str.title()

    # 5. Create a binary flag for logo presence
    df['has_logo'] = df['event_logo_url'].notna().astype(int)

    # 6. Extract year and month from start date
    df['event_year'] = df['event_start_date'].dt.year
    df['event_month'] = df['event_start_date'].dt.month

    # 7. Create a unique identifier for each event
    df['event_id'] = df.index

    # 8. Use LLM to tag event industry
    # df['event_industry'] = df.apply(lambda row: llm_tag_event_industry(row['event_name'], row['event_description']), axis=1)

    # 9. Extract domain from event_url
    df['event_domain'] = df['event_url'].str.extract(r'https?://(?:www\.)?([^/]+)')

    # 10. Create a flag for virtual events
    df['is_virtual'] = df['event_venue'].str.contains('virtual|online', case=False, na=False).astype(int)

    return df

# Usage
preprocessed_df = preprocess_event_data(csv_file_paths[1])
preprocessed_df.to_csv('preprocessed_event_info.csv', index=False)

print(preprocessed_df.head())
print(preprocessed_df.info())

                                      event_logo_url  \
0  http://expo.overseaseducation.sg/v.fastcdn.co/...   
1  https://ahiceconference.com/southeastasia/wp-c...   
2  https://apac.commoditytradingweek.com/wp-conte...   
3  https://architecturebuildingservices.com.sg/wp...   
4  https://asiandownstreaminsights.com/wp-content...   

                                    event_name event_start_date  \
0               World University Expo @ SUNTEC       2024-07-06   
1                        AHICE South East Asia       2025-02-25   
2                  Commodity Trading Week APAC       2025-02-25   
3  Architecture & Building Services (ABS) 2024       2024-11-13   
4                      Asian Downstream Summit       2024-10-23   

  event_end_date                                        event_venue  \
0     2024-07-06                           SUNTEC Convention Centre   
1     2025-02-26                              Pan Pacific Singapore   
2     2025-02-26                               

In [ ]:
def add_country_to_event_data(file_path):
  df = pd.read_csv(file_path)
  df['event_country'] = df.apply(lambda x : llm_event_country(x['event_venue'],x['event_url']),axis=1)
  df['event_country'] = df['event_country'].str.title()
  df['event_duration_in_days'] = df['event_duration']
  df.drop("event_duration",axis=1)
  return df

p_df = add_country_to_event_data("preprocessed_event_info.csv")
p_df.head()

,event_logo_url,event_name,event_start_date,event_end_date,event_venue,event_country,event_description,event_url,event_duration,has_logo,event_year,event_month,event_id,event_industry,event_domain,is_virtual,event_duration_in_days
0,http://expo.overseaseducation.sg/v.fastcdn.co/...,World University Expo @ SUNTEC,2024-07-06,2024-07-06,SUNTEC Convention Centre,Singapore,"Join our World University Expo this Saturday, ...",http://expo.overseaseducation.sg/,1.0,1,2024,7,0,Education,expo.overseaseducation.sg,0,1.0
1,https://ahiceconference.com/southeastasia/wp-c...,AHICE South East Asia,2025-02-25,2025-02-26,Pan Pacific Singapore,Singapore,The Asian Hotel Industry Conference & Exhibiti...,https://ahiceconference.com/southeastasia/,2.0,1,2025,2,1,Hospitality (Note: This is a distinct category...,ahiceconference.com,0,2.0
2,https://apac.commoditytradingweek.com/wp-conte...,Commodity Trading Week APAC,2025-02-25,2025-02-26,Marina Bay Sands,Singapore,Commodity Trading Week APAC is the premier eve...,https://apac.commoditytradingweek.com/,2.0,1,2025,2,2,Finance,apac.commoditytradingweek.com,0,2.0
3,https://architecturebuildingservices.com.sg/wp...,Architecture & Building Services (ABS) 2024,2024-11-13,2024-11-15,Marina Bay Sands Expo & Convention Centre Sing...,Singapore,The 10th Architecture & Building Services (ABS...,https://architecturebuildingservices.com.sg/,3.0,1,2024,11,3,Business,architecturebuildingservices.com.sg,0,3.0
4,https://asiandownstreaminsights.com/wp-content...,Asian Downstream Summit,2024-10-23,2024-10-24,"Sands Expo & Convention Centre, Singapore",Singapore,The Asian Downstream Summit is Asia’s most inf...,https://asiandownstreaminsights.com/events/asi...,2.0,1,2024,10,4,Energy,asiandownstreaminsights.com,0,2.0


In [ ]:
p_df.to_csv('preprocessed_event_info_1.csv', index=False)

# People Data Processing

In [ ]:
def llm_generate_email(first_name, last_name, middle_name, company_domain, email_pattern):
    # Check if the email pattern is empty

    try:
      email_pattern = str(email_pattern).lower()
      if not email_pattern or email_pattern=="nan":
          return "no email"  # Return "no email" for empty patterns


      # Extract the first initial and last initial
      first_initial = first_name[0].lower()
      last_initial = last_name[0].lower() if isinstance(last_name,str) else middle_name[0].lower()

      # Replace placeholders in the email pattern
      email = email_pattern.replace('[first]', first_name.lower())
      email = email.replace('[last]', last_name.lower() if isinstance(last_name,str) else middle_name.lower())
      email = email.replace('[first_initial]', first_initial)
      email = email.replace('[last_initial]', last_initial)

      # Add the company domain
      email += f"@{company_domain}"

      return email
    except Exception as e:
      return "no email"

In [ ]:
people_df = pd.read_csv(csv_file_paths[0])
people_df["email_pattern"]

0                         NaN
1                         NaN
2              [first].[last]
3                         NaN
4       [first_initial][last]
                ...          
6455           [first].[last]
6456           [first].[last]
6457                      NaN
6458                      NaN
6459                  [first]
Name: email_pattern, Length: 6460, dtype: object

In [ ]:
def preprocess_people_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # 1. Handle missing values
    df['middle_name'] = df['middle_name'].fillna('')
    df['person_city'] = df['person_city'].fillna('Unknown')
    df['person_state'] = df['person_state'].fillna('Unknown')
    df['person_country'] = df['person_country'].fillna('Unknown')

    # 2. Standardize names
    for col in ['first_name', 'middle_name', 'last_name']:
        df[col] = df[col].str.title()

    # 3. Create full name column
    df['full_name'] = df['first_name'] + ' ' + df['middle_name'] + ' ' + df['last_name']
    df['full_name'] = df['full_name'].str.replace('  ', ' ').str.strip()

    # 4. Standardize job titles
    df['job_title'] = df['job_title'].str.title()

    # 5. Standardize location information
    for col in ['person_city', 'person_state', 'person_country']:
        df[col] = df[col].str.title()

    df['email_pattern'] = df['email_pattern'].str.title()

    # 6. Generate email addresses
    df['generated_email'] = df.apply(lambda row: llm_generate_email(row['first_name'], row['last_name'],row['middle_name'],row['homepage_base_url'], row['email_pattern']), axis=1)

    # 7. Parse duration strings
    def parse_duration(duration_str):
        if pd.isna(duration_str):
            return np.nan
        years = re.search(r'(\d+)\s*years?', duration_str)
        months = re.search(r'(\d+)\s*months?', duration_str)
        total_months = 0
        if years:
            total_months += int(years.group(1)) * 12
        if months:
            total_months += int(months.group(1))
        return total_months

    df['duration_in_current_job_months'] = df['duration_in_current_job'].apply(parse_duration)
    df['duration_in_current_company_months'] = df['duration_in_current_company'].apply(parse_duration)

    # 8. Create a unique identifier for each person
    df['person_id'] = df.index

    # 9. Extract domain from homepage_base_url
    df['company_domain'] = df['homepage_base_url']

    # 10. Categorize job levels
    def categorize_job_level(title):
        title = str(title).lower()
        if any(role in title for role in ['ceo', 'cfo', 'cto', 'president', 'founder']):
            return 'Executive'
        elif any(role in title for role in ['director', 'head', 'lead','leader']):
            return 'Senior Management'
        elif any(role in title for role in ['manager', 'supervisor']):
            return 'Middle Management'
        else:
            return 'Individual Contributor'

    df['job_level'] = df['job_title'].apply(categorize_job_level)

    return df

# Usage
preprocessed_df = preprocess_people_data(csv_file_paths[0])
preprocessed_df.to_csv('preprocessed_people_info.csv', index=False)

print(preprocessed_df.head())
print(preprocessed_df.info())

  first_name middle_name last_name                       job_title  \
0    Cynthia               Battini                  Indirect Buyer   
1  Alexander               Mcclure                Public Relations   
2      Nadir                Shaari      Seo Junior Account Manager   
3    Daniela               Bonvino  Dilmah Sales Executive Manager   
4     Nikita                Kushwa    Business Development Manager   

  person_city person_state person_country          email_pattern  \
0     Unknown      Unknown         France                    NaN   
1      Austin           Tx             Us                    NaN   
2      London      Unknown             Gb         [First].[Last]   
3     Unknown      Unknown          Chile                    NaN   
4     Unknown      Unknown          India  [First_Initial][Last]   

   homepage_base_url duration_in_current_job duration_in_current_company  \
0       ariane.group                     NaN                         NaN   
1         amazon.c